In [ ]:
%pylab inline
from plot_all import plot_std, plot_images, semilogx_std
from pmap.mmd import logMMD

In [ ]:
import os
folder = "../paper/figs"
if not os.path.exists(folder):
    os.makedirs(folder, exist_ok=True)

# All results have been copied to
results_folder = "../results"

# Section 5.3

In [ ]:
a = np.load(f"../data/noisy_mnist.npz")
X_train = a["X_train"]
y_train = a["y_train"]
S = (X_train[y_train == 0] == 0).astype(np.float32).reshape(-1, X_train.shape[1] * X_train.shape[2])

learn_iter = 200
n_steps_sweep = np.array([5, 10, 25, 50, 100]) 
seeds = np.array([40, 41, 42, 43, 44]) 
logmmd = {}
sampling_algs = ["pmap","gibbs", "gwg", "gibbs_reset", "gwg_reset"]
for sampling_alg in sampling_algs:
    logmmd[sampling_alg] = np.zeros((len(seeds), len(n_steps_sweep)))
for i, seed in enumerate(seeds):
    for j, n_steps in enumerate(n_steps_sweep):
        for sampling_alg in sampling_algs:
            a = load(f"{results_folder}/zeros_ising/zeros_ising_samples_{learn_iter}_{sampling_alg}_nsteps_{n_steps}_seed_{seed}.npz")
            logmmdval, W, b, S2 = a["logmmd"], a["W"], a["b"], a["S2"]
            logmmd[sampling_alg][i,j] = logmmdval
            

# Get samples 
learn_iter, n_steps, seed = 200, 50, 40
sampling_alg="pmap"
S2pmap = load(f"{results_folder}/zeros_ising/zeros_ising_samples_{learn_iter}_{sampling_alg}_nsteps_{n_steps}_seed_{seed}.npz")["S2"]
sampling_alg="gibbs"
S2gibbs = load(f"{results_folder}/zeros_ising/zeros_ising_samples_{learn_iter}_{sampling_alg}_nsteps_{n_steps}_seed_{seed}.npz")["S2"]
sampling_alg="gibbs_reset"
S2gibbs_reset = load(f"{results_folder}/zeros_ising/zeros_ising_samples_{learn_iter}_{sampling_alg}_nsteps_{n_steps}_seed_{seed}.npz")["S2"]
sampling_alg="gwg"
S2gwg = load(f"{results_folder}/zeros_ising/zeros_ising_samples_{learn_iter}_{sampling_alg}_nsteps_{n_steps}_seed_{seed}.npz")["S2"]
sampling_alg="pmap_lp"
S2pmaplp = load(f"{results_folder}/zeros_ising/zeros_ising_samples_{learn_iter}_{sampling_alg}_nsteps_{n_steps}_seed_{seed}.npz")["S2"]


splot = vstack((S[8:12],S2pmap[8:12],S2gwg[8:12],S2gibbs_reset[8:12],S2gibbs[8:12],S2pmaplp.round()[8:12]))

In [ ]:
plot_images(splot.reshape(-1,30,30), nr=6)
ylabel("PMapLP  Gibbs   G-reset   GWG     PMP      Data  ", fontsize=22)
gca().yaxis.set_ticks([]) 
gca().xaxis.set_ticks([])
gca().spines['top'].set_visible(False)
gca().spines['bottom'].set_visible(False)
savefig("../paper/figs/zeros_ising.png", bbox_inches='tight')

# Section 5.2

In [ ]:
figure(figsize=(10*4/3,4))

a = np.load(f"{results_folder}/mplp/c2dlattice_mplp.npz")
logZmp, logZmplp, logZexact = a["logZmp"], a["logZmplp"], a["logZexact"]
sweep_d = [5, 10, 15]
sweep_side = [5, 10, 15]

logZmp -= logZexact
logZmplp -= logZexact

subplot(131)
plot_std(sweep_side, logZmp.mean(0), logZmp.std(0)/sqrt(logZmp.shape[0]))
plot_std(sweep_side, logZmplp.mean(0), logZmplp.std(0)/sqrt(logZmplp.shape[0]),'orange')
plot(sweep_side,logZmplp.mean(0),":")
xlabel("Side of cyclic lattice",fontsize=15)
ylabel("$\log(Z_{approx}) - \log(Z_{exact})$",fontsize=16)
legend(["PMP","MPLP (500x slower)"],fontsize=14)
title("Log-partition on cyc. 2D lattice",fontsize=14)
# savefig("../paper/figs/fig_mplp_zeros.pdf",bbox_inches='tight')

a = np.load(f"{results_folder}/mplp/ising_mplp.npz")
logZmp, logZmplp, logZexact = a["logZmp"], a["logZmplp"], a["logZexact"]

logZmp -= logZexact
logZmplp -= logZexact

subplot(132)
plot_std(sweep_d, logZmp.mean(0), logZmp.std(0)/sqrt(logZmp.shape[0]))
plot_std(sweep_d, logZmplp.mean(0), logZmplp.std(0)/sqrt(logZmplp.shape[0]),'orange')
xlabel("Dimensions of Ising model",fontsize=15)
legend(["PMP","MPLP (500x slower)"],fontsize=14)
title("Log-partition on Ising model",fontsize=14)

subplot(133)
colors=["blue","orange","green","purple","black", "red"]
for sampling_alg, color in zip(sampling_algs, colors):
    plot_std(n_steps_sweep, logmmd[sampling_alg].mean(0), logmmd[sampling_alg].std(0)/sqrt(len(seeds)),color)
legend(["PMP", "Gibbs (10x slower)", "GWG (30x slower)", 
        "Gibbs-reset (10x slower)", "GWG-reset (30x slower)", "PMapLP (1400x slower)"],fontsize=11,loc=(0.25,0.51))
xlabel("# Full sweeps (sampling steps)",fontsize=15)
ylabel("$\log(MMD^2)$",fontsize=16)
title("Ising model trained on 'zeros'",fontsize=14)
plt.tight_layout()
savefig("../paper/figs/mplp_zeros_ising.pdf",bbox_inches='tight')

# Section 5.4

In [ ]:
learn_iter = 1000
theta, side = -0.1, 25
n_steps_sweep = np.array([5, 10, 25, 50, 100]) 
seeds = np.array([40, 41, 42, 43, 44]) 
logRMSE = {}
logmmd = {}
sampling_algs = ["pmap","gibbs", "gwg"]
for sampling_alg in sampling_algs:
    logRMSE[sampling_alg] = np.zeros((len(seeds), len(n_steps_sweep)))
    logmmd[sampling_alg] = np.zeros((len(seeds), len(n_steps_sweep)))
for i, seed in enumerate(seeds):
    for j, n_steps in enumerate(n_steps_sweep):
        for sampling_alg in sampling_algs:
            a = load(f"{results_folder}/c2d_lattice/c2dlat_slow_li_{learn_iter}_{sampling_alg}_theta_{theta}_side_{side}_nsteps_{n_steps}_seed_{seed}.npz")
            logmmdval, logrmse, Wgt, bgt, W, b, S2 = a["logmmd"],a["logrmse"], a["Wgt"], a["bgt"], a["W"], a["b"], a["S2"]
            logRMSE[sampling_alg][i,j] = logrmse
            logmmd[sampling_alg][i,j] = logmmdval
logRMSE1,logmmd1 = logRMSE, logmmd
            
            
learn_iter = 100000
theta, side = -0.1, 25
n_steps_sweep = np.array([5, 10, 25, 50, 100]) 
seeds = np.array([40, 41, 42, 43, 44]) 
logRMSE = {}
logmmd = {}
sampling_algs = ["pmap","gibbs", "gwg"]
for sampling_alg in sampling_algs:
    logRMSE[sampling_alg] = np.zeros((len(seeds), len(n_steps_sweep)))
    logmmd[sampling_alg] = np.zeros((len(seeds), len(n_steps_sweep)))
for i, seed in enumerate(seeds):
    for j, n_steps in enumerate(n_steps_sweep):
        for sampling_alg in sampling_algs:
            a = load(f"{results_folder}/c2d_lattice/c2dlat_li_{learn_iter}_{sampling_alg}_theta_{theta}_side_{side}_nsteps_{n_steps}_seed_{seed}.npz")
            logmmdval, logrmse, Wgt, bgt, W, b, S2 = a["logmmd"],a["logrmse"], a["Wgt"], a["bgt"], a["W"], a["b"], a["S2"]
            logRMSE[sampling_alg][i,j] = logrmse
            logmmd[sampling_alg][i,j] = logmmdval
        
logRMSE2,logmmd2 = logRMSE, logmmd

learn_iter = 100000
n_steps_sweep = np.array([5, 10, 25, 50, 100]) 
seeds = np.array([40, 41, 42, 43, 44]) 
logRMSE = {}
logmmd = {}
sampling_algs = ["pmap","gibbs", "gwg"]
for sampling_alg in sampling_algs:
    logRMSE[sampling_alg] = np.zeros((len(seeds), len(n_steps_sweep)))
    logmmd[sampling_alg] = np.zeros((len(seeds), len(n_steps_sweep)))
for i, seed in enumerate(seeds):
    for j, n_steps in enumerate(n_steps_sweep):
        for sampling_alg in sampling_algs:
            a = load(f"{results_folder}/erdos/erdos_li_{learn_iter}_{sampling_alg}_nsteps_{n_steps}_seed_{seed}.npz")
            logmmdval, logrmse, Wgt, bgt, W, b, S2 = a["logmmd"],a["logrmse"], a["Wgt"], a["bgt"], a["W"], a["b"], a["S2"]
            logRMSE[sampling_alg][i,j] = logrmse
            logmmd[sampling_alg][i,j] = logmmdval
        
logRMSE3,logmmd3 = logRMSE, logmmd

In [ ]:
figure(figsize=(10*4/3,4))
subplot(131)
colors=["blue","orange","green"]
for sampling_alg, color in zip(sampling_algs, colors):
    plot_std(n_steps_sweep, logmmd1[sampling_alg].mean(0),logmmd1[sampling_alg].std(0)/sqrt(len(seeds)),color)
legend(["PMP","Gibbs-reset (150x)", "GWG-reset (800x)"],fontsize=12)
ylabel("$\log(RMSE)$",fontsize=16)
xlabel("Learning and sampling, # full sweeps",fontsize=15)
ylim([-8.25,-7.5])
title("Cyc. 2D lattice",fontsize=14)
      
subplot(132)
colors=["blue","orange","green"]
for sampling_alg, color in zip(sampling_algs, colors):
    plot_std(n_steps_sweep, logmmd2[sampling_alg].mean(0), logmmd2[sampling_alg].std(0)/sqrt(len(seeds)),color)
legend(["PMP","Gibbs", "GWG"],fontsize=12)
xlabel("Learning, # MCMC steps or equiv",fontsize=15)
title("Cyc. 2D lattice",fontsize=14)
    
subplot(133)
colors=["blue","orange","green"]
for sampling_alg, color in zip(sampling_algs, colors):
    plot_std(n_steps_sweep, logmmd3[sampling_alg].mean(0), logmmd3[sampling_alg].std(0)/sqrt(len(seeds)),color)
legend(["PMP","Gibbs", "GWG"],fontsize=12)
xlabel("Learning, # MCMC steps or equiv",fontsize=15)
title("Erdos-Renyi model",fontsize=14)
plt.tight_layout()
savefig("../paper/figs/erdoslatt_mmd.pdf", bbox_inches='tight')

In [ ]:
figure(figsize=(10*4/3,4))
subplot(131)
colors=["blue","orange","green"]
for sampling_alg, color in zip(sampling_algs, colors):
    plot_std(n_steps_sweep, logRMSE1[sampling_alg].mean(0),logRMSE1[sampling_alg].std(0)/sqrt(len(seeds)),color)
plot(n_steps_sweep, logRMSE1["gibbs"].mean(0),":",color='orange')
legend(["PMP","Gibbs-reset (150x)", "GWG-reset (800x)"],fontsize=12)
ylabel("$\log(MMD^2)$",fontsize=16)
xlabel("Learning and sampling, # full sweeps",fontsize=15)
title("Cyc. 2D lattice",fontsize=14)
      
subplot(132)
colors=["blue","orange","green"]
for sampling_alg, color in zip(sampling_algs, colors):
    plot_std(n_steps_sweep, logRMSE2[sampling_alg].mean(0),logRMSE2[sampling_alg].std(0)/sqrt(len(seeds)),color)
legend(["PMP","Gibbs", "GWG"],fontsize=12)
xlabel("Learning, # MCMC steps or equiv",fontsize=15)
title("Cyc. 2D lattice",fontsize=14)

subplot(133)
colors=["blue","orange","green"]
for sampling_alg, color in zip(sampling_algs, colors):
    plot_std(n_steps_sweep, logRMSE3[sampling_alg].mean(0),logRMSE3[sampling_alg].std(0)/sqrt(len(seeds)),color)
legend(["PMP","Gibbs", "GWG"],fontsize=12)
xlabel("Learning, # MCMC steps or equiv",fontsize=15) 
title("Erdos-Renyi model",fontsize=14)
plt.tight_layout()
savefig("../paper/figs/erdoslatt_rmse.pdf", bbox_inches='tight')

# Section 5.5

In [ ]:
learn_iter = 120000  # with 100 MCMC steps in learning and 0.01 learning rate (200 epochs) (just pmap)
n_steps_sweep = np.array([5, 10, 25, 50, 100, 200, 500, 1000, 2000])
seeds = np.array([40, 41, 42, 43, 44]) 
logmmd = {}
sampling_algs = ["pmap", "gibbs", "gibbs_reset", "gibbs1"]
for sampling_alg in sampling_algs:
    logmmd[sampling_alg] = np.zeros((len(seeds), len(n_steps_sweep)))
for i, seed in enumerate(seeds):
    for j, n_steps in enumerate(n_steps_sweep):
        for sampling_alg in sampling_algs:
            a = load(f"{results_folder}/rbm_mnist/rbm_mnist_noadam_samples_{learn_iter}_{sampling_alg}_nsteps_{n_steps}_seed_{seed}.npz")
            logmmdval, S2 = a["logmmd"], a["S2"]
            logmmd[sampling_alg][i,j] = logmmdval

In [ ]:
colors=["blue","orange","green","purple","black"]
for sampling_alg, color in zip(sampling_algs, colors):
    semilogx_std(n_steps_sweep, logmmd[sampling_alg].mean(0), logmmd[sampling_alg].std(0)/sqrt(len(seeds)),color)
legend(["PMP", "PCD-100 (3x faster)", "Gibbs-reset (3x faster)", "PCD-1 (300x faster)"],fontsize=11,loc=(0.55,0.15))
xlabel("# Full sweeps (sampling steps)",fontsize=15)
ylabel("$\log(MMD^2)$",fontsize=16)
savefig("../paper/figs/rbm_graph.pdf", bbox_inches='tight')

In [ ]:
seed=40
n_steps = 100

sampling_alg="pmap"
a = np.load(f"{results_folder}/rbm_mnist/rbm_mnist_noadam_samples_{learn_iter}_{sampling_alg}_nsteps_{n_steps}_seed_{seed}.npz")
S2_pmap  =a["S2"]
sampling_alg="gibbs_reset"
a = np.load(f"{results_folder}/rbm_mnist/rbm_mnist_noadam_samples_{learn_iter}_{sampling_alg}_nsteps_{n_steps}_seed_{seed}.npz")
S2_gibbs_reset=  a["S2"]
sampling_alg="gibbs"
a = np.load(f"{results_folder}/rbm_mnist/rbm_mnist_noadam_samples_{learn_iter}_{sampling_alg}_nsteps_{n_steps}_seed_{seed}.npz")
S2_gibbs = a["S2"]
sampling_alg="gibbs1"
a = np.load(f"{results_folder}/rbm_mnist/rbm_mnist_noadam_samples_{learn_iter}_{sampling_alg}_nsteps_{n_steps}_seed_{seed}.npz")
S2_gibbs1 = a["S2"]

N=10
a = np.load("../data/mnist_digits_and_labels.npz")
S = (a["X_train"].reshape(-1, 28 ** 2)>0.5).astype(np.float32)
splot = vstack((S[:N],S2_pmap[:N],S2_gibbs_reset[:N],S2_gibbs[:N],S2_gibbs1[:N]))


In [ ]:
plot_images(splot.reshape(-1,28,28), nr=5)
ylabel("PCD1 PCD100 G-reset PMP    Data",fontsize=16)
gca().yaxis.set_ticks([]) 
gca().xaxis.set_ticks([]) 
savefig("../paper/figs/rbm_samples.png", bbox_inches='tight')

# Section 5.5 - 2s only

In [ ]:
learn_iter = 1000  # with 100 MCMC steps in learning and 0.01 learning rate (200 epochs) (just pmap)
n_steps_sweep = np.array([100])
seeds = np.array([40, 41, 42, 43, 44]) 
logmmd = {}
sampling_algs = ["pmap", "gibbs", "gibbs_reset", "gibbs1", "pmap_lp"]
for sampling_alg in sampling_algs:
    logmmd[sampling_alg] = np.zeros((len(seeds), len(n_steps_sweep)))
for i, seed in enumerate(seeds):
    for j, n_steps in enumerate(n_steps_sweep):
        for sampling_alg in sampling_algs:
            a = load(f"{results_folder}/rbm_mnist/rbm_2s_adam_samples_{learn_iter}_{sampling_alg}_nsteps_{n_steps}_seed_{seed}.npz")
            logmmdval, S2 = a["logmmd"], a["S2"]
            logmmd[sampling_alg][i,j] = logmmdval

for sampling_alg in sampling_algs:
    print(sampling_alg, logmmd[sampling_alg].mean(0), logmmd[sampling_alg].std(0)/sqrt(len(seeds)))

In [ ]:
seed=40
n_steps = 100

sampling_alg="pmap"
a = np.load(f"{results_folder}/rbm_mnist/rbm_2s_adam_samples_{learn_iter}_{sampling_alg}_nsteps_{n_steps}_seed_{seed}.npz")
S2_pmap  =a["S2"]
sampling_alg="gibbs_reset"
a = np.load(f"{results_folder}/rbm_mnist/rbm_2s_adam_samples_{learn_iter}_{sampling_alg}_nsteps_{n_steps}_seed_{seed}.npz")
S2_gibbs_reset=  a["S2"]
sampling_alg="gibbs"
a = np.load(f"{results_folder}/rbm_mnist/rbm_2s_adam_samples_{learn_iter}_{sampling_alg}_nsteps_{n_steps}_seed_{seed}.npz")
S2_gibbs = a["S2"]
sampling_alg="gibbs1"
a = np.load(f"{results_folder}/rbm_mnist/rbm_2s_adam_samples_{learn_iter}_{sampling_alg}_nsteps_{n_steps}_seed_{seed}.npz")
S2_gibbs1 = a["S2"]
sampling_alg="pmap_lp"
a = np.load(f"{results_folder}/rbm_mnist/rbm_2s_adam_samples_{learn_iter}_{sampling_alg}_nsteps_{n_steps}_seed_{seed}.npz")
S2_pmaplp = a["S2"]

N=10
a = np.load("../data/mnist_digits_and_labels.npz")
S = (a["X_train"][a["y_train"] == 2].reshape(-1, 28 ** 2)>0.5).astype(np.float32)
splot = vstack((S[:N],S2_pmap[:N],S2_gibbs_reset[:N],S2_gibbs[:N],S2_gibbs1[:N],S2_pmaplp[:N]))


In [ ]:
plot_images(splot.reshape(-1,28,28), nr=6)
ylabel("PmapLP  PCD1   PCD100  G-reset  PMP    Data ",fontsize=14)
gca().yaxis.set_ticks([]) 
gca().xaxis.set_ticks([]) 
savefig("../paper/figs/rbm_2s.png", bbox_inches='tight')

# Section 5.6

In [ ]:
W = np.array([np.load(f"{results_folder}/convor/convor_solution_seed_{seed}.npz")["W"][0] for seed in [40,41,42,43,44]])
splot = W.reshape(-1,6,6)
big_image1 = plot_images(np.heaviside(splot,0.5),nr=5, display=False)
big_image1[:,::6+1] = [0.502,0,0.502]
big_image1 = np.pad(big_image1,((1,0),(5,5),(0,0)),constant_values=1)

X = np.load("../data/conv_problem.npz")['X']
big_image = plot_images(X[:3,0], nr=1, display=False)
big_image[:,::14+1] = [0.502,0,0.502]

plt.figure(figsize=(10, 10))
plt.imshow(np.vstack((big_image,big_image1)), interpolation="none")
gca().axis("off")
savefig("../paper/figs/convor.png", bbox_inches='tight')

In [ ]:
X = np.load("../data/conv_problem.npz")['X']
big_image = plot_images(X[:9,0], display=False)
big_image[:,::14+1] = [0.502,0,0.502]

plt.figure(figsize=(10, 10))
plt.imshow(big_image, interpolation="none")
gca().axis("off")
savefig("../paper/figs/convor_1.png", bbox_inches='tight')

In [ ]:
W = np.array([np.load(f"{results_folder}/convor/convor_solution_seed_{seed}.npz")["W"][0] for seed in [40,41,42,43,44]])
splot = W.reshape(-1,6,6)
big_image1 = plot_images(np.heaviside(splot,0.5),nr=5)
gca().axis("off")
savefig("../paper/figs/convor_2.png", bbox_inches='tight')